NOTE: This notebook is assuming you have the `daily_dataset.csv` inside the folder local_data in data, and the `pernoctacions_2019_2024.csv`, `temperature_precipitation.csv` and `tourism_flux.csv` datasets inside the folder data.

The `daily_dataset.csv` should be exactly how it is when it is being recently downloaded from the AB Datachallenge portal.

In [1]:
import pandas as pd
from collections import Counter
import seaborn as sns
import os
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


## 1. Preprocessing of daily_dataset

In [2]:
# Step 1: Load and preprocess daily water consumption data
daily_aigues = pd.read_csv("../data/local_data/daily_dataset.csv", encoding='latin1')

# Rename columns for clarity
daily_aigues = daily_aigues.rename(columns={
    daily_aigues.columns[0]: 'Census Section',
    daily_aigues.columns[1]: 'District',
    daily_aigues.columns[2]: 'Municipality',
    daily_aigues.columns[3]: 'Date',
    daily_aigues.columns[4]: 'Use',
    daily_aigues.columns[5]: 'Number of Meters',
    daily_aigues.columns[6]: 'Accumulated Consumption'
})

# Convert Date column to datetime
daily_aigues['Date'] = pd.to_datetime(daily_aigues['Date'])

# Replace values in the 'Use' column
daily_aigues['Use'] = daily_aigues['Use'].replace({
    'Comercial/Comercial/Commercial': 'Commercial',
    'DomÃ¨stic/DomÃ©stico/Domestic': 'Domestic',
    'Industrial/Industrial/Industrial': 'Industrial'
})

# Filter data for Barcelona
daily_aigues_bcn = daily_aigues[daily_aigues['Municipality'] == 'BARCELONA'].drop(columns=["Municipality"])

# Drop rows with invalid Census Section or District values
daily_aigues_bcn = daily_aigues_bcn[daily_aigues_bcn['Census Section'] != '<NULL>']
daily_aigues_bcn = daily_aigues_bcn[daily_aigues_bcn['District'] != '>']

# Remove rows with Accumulated Consumption values below 0
daily_aigues_bcn = daily_aigues_bcn[daily_aigues_bcn['Accumulated Consumption'] > 0]



## 2. Adding meteorological data

In [3]:
# Step 2: Adding meteorological data (temperature and precipitation)
clima_df = pd.read_csv('../data/temperature_precipitation.csv')

# Convert 'fecha' column to datetime and group by date
clima_df['fecha'] = pd.to_datetime(clima_df['fecha'])
clima_df = clima_df.groupby('fecha').agg({
    'temp_max': 'mean',
    'temp_min': 'mean',
    'precipitacion': 'mean'
}).round(2).reset_index()

# Merge meteorological data with water consumption data
merged_data = daily_aigues_bcn.merge(clima_df, left_on='Date', right_on='fecha', how='left').drop(columns=['fecha'])

# Debug: Check the merged data
print("Merged data after adding meteorological data:")
print(merged_data.head())


Merged data after adding meteorological data:
  Census Section District       Date         Use  Number of Meters  \
0      801901001        1 2021-01-01  Commercial                56   
1      801901001        1 2021-01-01    Domestic               322   
2      801901001        1 2021-01-01  Industrial                10   
3      801901001        1 2021-01-02  Commercial                56   
4      801901001        1 2021-01-02    Domestic               322   

   Accumulated Consumption  temp_max  temp_min  precipitacion  
0                     2710      7.55      4.30           5.65  
1                    10058      7.55      4.30           5.65  
2                   140646      7.55      4.30           5.65  
3                     3573      8.55      1.55           0.00  
4                     9600      8.55      1.55           0.00  


## 3. Adding tourism data

In [ ]:
# Load and prepare the data
import pandas as pd

targetes = pd.read_csv('../data/dataset_targetes.csv')

targetes = targetes.drop('Espanyola', axis=1)

# Convert the column to datetime format
targetes['Data'] = pd.to_datetime(targetes['Data'])

# Format the date as yyyy-mm-dd
targetes['Data'] = targetes['Data'].dt.strftime('%Y-%m-%d')

# Filter the DataFrame to include only dates from 2021 to 2023
targetes = targetes[(targetes['Data'] >= '2021-01-01') & (targetes['Data'] <= '2023-12-31')]

transactions = pd.read_csv('../data/total_transactions.csv')

# Convert the column to datetime format
transactions['Data'] = pd.to_datetime(transactions['Data'])

# Format the date as yyyy-mm-dd
transactions['Data'] = transactions['Data'].dt.strftime('%Y-%m-%d')

# Filter the DataFrame to include only dates from 2021 to 2023
transactions = transactions[(transactions['Data'] >= '2021-01-01') & (transactions['Data'] <= '2023-12-31')]

# Set the date column as the index for both DataFrames to align dates easily
transactions.set_index('Data', inplace=True)
targetes.set_index('Data', inplace=True)

# Create the total_tourist_transactions column by multiplying Series 1 in transactions by Estrangera in targetes
targetes['total_tourist_transactions'] = (targetes['Estrangera'] * transactions['Series 1'])/100
targetes.reset_index(inplace=True)
transactions = targetes.drop('Estrangera', axis=1)

# Load the datasets
pernoctacions = pd.read_csv('../data/pernoctacions_2019_2024.csv')

# Convert date columns to datetime
pernoctacions['Data'] = pd.to_datetime(pernoctacions['Data'], format='%m/%d/%Y')
transactions['Data'] = pd.to_datetime(transactions['Data'], format='%Y-%m-%d')

# Clean and convert the 'Pernoctacions' column to numeric
pernoctacions['Pernoctacions'] = pd.to_numeric(pernoctacions['Pernoctacions'].str.replace('.', ''), errors='coerce')

# Group transactions by month and calculate the total transactions per month
monthly_transactions = transactions.groupby(transactions['Data'].dt.to_period('M'))['total_tourist_transactions'].sum().reset_index()
monthly_transactions.columns = ['Month', 'Total Transactions']

# Merge monthly totals with daily transactions to calculate daily percentages
transactions = transactions.merge(monthly_transactions, left_on=transactions['Data'].dt.to_period('M'), right_on='Month', how='left')
transactions['Percentage of Transactions'] = transactions['total_tourist_transactions'] / transactions['Total Transactions']

# Remove unnecessary column
transactions.drop(columns=['Month'], inplace=True)

# Initialize the results list
resultados = []

# Loop through each row in pernoctacions to distribute monthly totals across days
for _, row in pernoctacions.iterrows():
    mes = row['Data'].month
    anyo = row['Data'].year
    pernoctaciones_mensuales = row['Pernoctacions']
    
    # Filter the transactions for the current month and year
    transactions_mes = transactions[(transactions['Data'].dt.month == mes) & (transactions['Data'].dt.year == anyo)]
    
    # Ensure the daily percentages add up to 1
    transactions_mes['Percentage of Transactions'] /= transactions_mes['Percentage of Transactions'].sum()
    
    # Distribute pernoctacions based on daily percentages
    for _, transaction in transactions_mes.iterrows():
        pernoctaciones_diarias = round(pernoctaciones_mensuales * transaction['Percentage of Transactions'])
        resultados.append({
            'Day': transaction['Data'],
            'tipus allotjament': row['Tipologia d\'allotjament'],
            'pernoctacions': pernoctaciones_diarias
        })

# Create the final DataFrame
resultado_df = pd.DataFrame(resultados)

# Display a sample of the final DataFrame

# Merge the water consumption data with the tourism data
final_data = merged_data.merge(resultado_df, left_on='Date', right_on='Day', how='left').drop(columns=['Day'])

# Grouping by all columns except 'tipus allotjament' and 'pernoctacions', and summing 'pernoctacions'
final_data = (
    final_data.groupby([
        "Census Section", "District", "Date", "Use", 
        "Number of Meters", "Accumulated Consumption", 
        "temp_max", "temp_min", "precipitacion"
    ], as_index=False)
    .agg({"pernoctacions": "sum"})
)

# Grouping by the relevant columns and summing the required fields
final_data = (
    final_data.groupby([
        "Census Section", "District", "Date", "temp_max", "temp_min", "precipitacion", "pernoctacions"
    ], as_index=False)
    .agg({
        "Number of Meters": "sum",
        "Accumulated Consumption": "sum"
    })
)

# Drop the number of meters column
final_data.drop(columns=["Number of Meters"], inplace=True)

In [ ]:
final_data.head(10)

,Census Section,District,Date,temp_max,temp_min,precipitacion,pernoctacions,Number of Meters,Accumulated Consumption
0,801901001,1,2021-01-01,7.55,4.30,5.65,6186,388,153414
1,801901001,1,2021-01-02,8.55,1.55,0.00,16193,388,166201
2,801901001,1,2021-01-03,9.55,0.85,0.00,10984,388,129309
3,801901001,1,2021-01-04,10.10,0.65,0.00,15115,387,141909
4,801901001,1,2021-01-05,9.40,0.65,0.00,15962,387,137782
5,801901001,1,2021-01-06,9.70,0.25,0.50,9029,387,125862
6,801901001,1,2021-01-07,6.35,2.25,0.00,13770,387,117474
7,801901001,1,2021-01-08,6.10,0.50,5.10,15144,387,129696
8,801901001,1,2021-01-09,6.15,1.50,12.40,10331,387,146591
9,801901001,1,2021-01-10,6.65,3.60,6.00,7811,387,125036


## 4. Save the cleaned dataset

In [6]:
# Step 4: Save the cleaned dataset
# os.makedirs('../data/local_data/', exist_ok=True)
# final_data.to_csv('../data/local_data/merged_cleaned_data_NEW_p.csv', index=False)